In [ ]:
# !pip install pandarallel 

# import gc

# import numpy as np
# import pandas as pd
# from pathlib import Path

# from pandarallel import pandarallel
# pandarallel.initialize()

# BASE_DIR = Path('../input/mlb-player-digital-engagement-forecasting')
# train = pd.read_csv(BASE_DIR / 'train.csv')

# null = np.nan
# true = True
# false = False

# for col in train.columns:

#     if col == 'date': continue

#     _index = train[col].notnull()
#     train.loc[_index, col] = train.loc[_index, col].parallel_apply(lambda x: eval(x))

#     outputs = []
#     for index, date, record in train.loc[_index, ['date', col]].itertuples():
#         _df = pd.DataFrame(record)
#         _df['index'] = index
#         _df['date'] = date
#         outputs.append(_df)

#     outputs = pd.concat(outputs).reset_index(drop=True)

#     outputs.to_csv(f'{col}_train.csv', index=False)
#     outputs.to_pickle(f'{col}_train.pkl')

#     del outputs
#     del train[col]
#     gc.collect()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.metrics import mean_absolute_error
from datetime import timedelta
from tqdm import tqdm
import mlb
import gc

In [ ]:
players = pd.read_csv("../input/mlb-player-digital-engagement-forecasting/players.csv")
players_stats = pd.read_csv("../input/extradata/players_stats.csv")
rosters = pd.read_pickle('../input/mlb-pdef-train-dataset/rosters_train.pkl')

In [ ]:
FECOLS = ['t1_mean', 't2_mean', 't3_mean', 't4_mean',
       't1_max', 't2_max', 't3_max', 't4_max', 't1_min', 't2_min', 't3_min',
       't4_min', 't1_std', 't2_std', 't3_std', 't4_std']
TGTCOLS = ['target1', 'target2', 'target3', 'target4']

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

In [ ]:
def make_model(n_in):
    inp = L.Input(name="inputs", shape=(n_in,))
    nh = 128
    x = L.Dense(nh, activation="relu", name="d1")(inp)
    x = L.Dense(nh, activation="relu", name="d2")(x)
    x = L.Dense(nh, activation="relu", name="d3")(x)
    preds = L.Dense(4, activation="linear", name="preds")(x)
    
    model = M.Model(inp, preds, name="ANN_XST")
    model.compile(loss="mean_absolute_error", optimizer="adam")
    return model

In [ ]:
XST = make_model(16)
print(XST.summary())

In [ ]:
XST.load_weights("../input/trainingxst/model_XST.cpkt")

In [ ]:
# #to debug submission process
# sample_pred_temp = pd.DataFrame(columns=['date_playerId', 'target1', 'target2', 'target3', 'target4'])
# test_temp = pd.DataFrame(columns=['date_playerId', 'playerId', 'gameDate', 'teamId', 'statusCode',
#        'status', 'bmi', 'DOB', 'age', 'is_played', 'Unnamed: 0', 't1_m',
#        't2_m', 't3_m', 't4_m'])

# Submission

In [ ]:
env = mlb.make_env() # initialize the environment
iter_test = env.iter_test() # iterator which loops over each date in test set

In [ ]:
null = np.nan
true = True
false = False

for (test_df, sample_prediction_df) in iter_test: # make predictions here
    
    sample_prediction_df = sample_prediction_df.reset_index(drop=True)
    
    # creat dataset
    sample_prediction_df['playerId'] = sample_prediction_df['date_playerId']\
                                        .map(lambda x: int(x.split('_')[1]))
    # Dealing with missing values
    if test_df['rosters'].iloc[0] == test_df['rosters'].iloc[0]:
        test_rosters = pd.DataFrame(eval(test_df['rosters'].iloc[0]))
    else:
        test_rosters = pd.DataFrame({'playerId': sample_prediction_df['playerId']})
        for col in rosters.columns:
            if col == 'playerId': continue
            test_rosters[col] = np.nan
            
    sub_df = sample_prediction_df[['playerId']].copy()
    sub_df = sub_df.merge(players_stats,on='playerId',how='left')
    
    X_fe = sub_df[FECOLS].fillna(0.).values
    preds = XST.predict(X_fe)
    preds = np.clip(preds, 0, 100)
    
    # merge submission
    sample_prediction_df[TGTCOLS] =  preds
    sample_prediction_df = sample_prediction_df.fillna(0.)
    
    final_headers = ["date_playerId", "target1", "target2", "target3", "target4"]
    sample_prediction_df = sample_prediction_df[final_headers].copy()
    #del sample_prediction_df['playerId']
    
#      #to debug
#     sample_pred_temp = sample_pred_temp.append(sample_prediction_df)#,ignore_index=True)
#     #test_temp = test_temp.append(test) #ignore_index=True)
#     #
    
    env.predict(sample_prediction_df)

In [ ]:
sample_prediction_df

In [ ]:
# # to reset env.predict() to work
# example_sample_submission = pd.read_csv("../input/mlb-player-digital-engagement-forecasting/example_sample_submission.csv")
# example_sample_submission
# #env.predict(example_sample_submission) 

In [ ]:
# # to check if any date_playerId is missing in submission
# list(example_sample_submission.date_playerId)==list(sample_pred_temp.date_playerId)


In [ ]:
# # to trace mismatch
# for id in zip(sample_pred_temp.date_playerId,example_sample_submission.date_playerId):
#     if id[0]==id[1]:
#         pass
#     else:
#         print("EXCEPTION")

In [ ]:
# sample_pred_temp.info()

In [ ]:
# example_sample_submission.info()

In [ ]:
# sample_pred_temp